# Feature Importance with Captum

In [1]:
import torch
from importlib import import_module

WORKSPACE_DIR = '/workspaces/msc-thesis-recurrent-health-modeling/'
print(f"Workspace directory: {WORKSPACE_DIR}")

# ---- User inputs (EDIT ME) ----
DATASET_TEST_PT_PATH = f"{WORKSPACE_DIR}_models/mimic/deep_learning/attention_pooling/multiple_hosp_patients/test_dataset.pt"  # path to test .pt file
DATASET_VAL_PT_PATH = f"{WORKSPACE_DIR}_models/mimic/deep_learning/attention_pooling/multiple_hosp_patients/validation_tuning_dataset.pt"  # path to val .pt file
DATASET_TRAIN_PT_PATH = f"{WORKSPACE_DIR}_models/mimic/deep_learning/attention_pooling/multiple_hosp_patients/train_tuning_dataset.pt"  # path to train .pt file
MODEL_MODULE = "recurrent_health_events_prediction.model.RecurrentHealthEventsDL"
MODEL_CLASS_NAME = "AttentionPoolingNet"   # e.g., GRUNet, AttentionPoolingNet, CrossAttnPoolingNet, AttentionPoolingNetCurrentQuery
CONFIG_PATH = f"{WORKSPACE_DIR}_runs/attention_pooling_20251106_181803/model_config.yaml" # path to model config yaml

Workspace directory: /workspaces/msc-thesis-recurrent-health-modeling/


In [2]:
TRAINING_DATA_STATS_PATH = f"{WORKSPACE_DIR}_models/mimic/deep_learning/training_stats.json"  # path to training stats json

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## Load datasets and model

In [4]:
if CONFIG_PATH is not None:
    import yaml
    with open(CONFIG_PATH, "r") as f:
        model_cfg = yaml.safe_load(f)
    MODEL_CLASS_NAME = model_cfg.get("model_class", MODEL_CLASS_NAME)
    MODEL_PARAMS = model_cfg.get("model_params")
    print("Loaded config:", CONFIG_PATH)
else:
    print("Using inline MODEL_PARAMS.")
print("Model class:", MODEL_CLASS_NAME)
print("Model params:", MODEL_PARAMS)


Loaded config: /workspaces/msc-thesis-recurrent-health-modeling/_runs/attention_pooling_20251106_181803/model_config.yaml
Model class: AttentionPoolingNet
Model params: {'dropout': 0.12112313358911685, 'hidden_size_head': 128, 'hidden_size_seq': 16, 'input_size_curr': 19, 'input_size_seq': 8}


In [5]:
model_cfg

{'batch_size': 32,
 'current_feat_cols': ['LOG_HOSPITALIZATION_DAYS',
  'LOG_DAYS_IN_ICU',
  'CHARLSON_INDEX',
  'LOG_NUM_DRUGS',
  'NUM_PROCEDURES',
  'LOG_PARTICIPATION_DAYS',
  'HAS_DIABETES',
  'HAS_COPD',
  'HAS_CONGESTIVE_HF',
  'DISCHARGE_LOCATION_POST_ACUTE_CARE',
  'DISCHARGE_LOCATION_HOME',
  'AGE',
  'GENDER_M',
  'ADMISSION_TYPE_ELECTIVE',
  'ETHNICITY_WHITE',
  'ETHNICITY_BLACK',
  'ETHNICITY_HISPANIC',
  'INSURANCE_MEDICAID',
  'INSURANCE_PRIVATE'],
 'learning_rate': 0.008273075604824992,
 'longitudinal_feat_cols': ['LOG_HOSPITALIZATION_DAYS',
  'LOG_DAYS_IN_ICU',
  'CHARLSON_INDEX',
  'LOG_NUM_DRUGS',
  'NUM_PROCEDURES',
  'DISCHARGE_LOCATION_POST_ACUTE_CARE',
  'ADMISSION_TYPE_ELECTIVE',
  'LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION'],
 'lr_scheduler': 'cosine',
 'max_sequence_length': 4,
 'model_class': 'AttentionPoolingNet',
 'model_name': 'Attention Pooling',
 'model_params': {'dropout': 0.12112313358911685,
  'hidden_size_head': 128,
  'hidden_size_seq': 16,
  'input_size_

In [6]:
current_feat_cols = model_cfg.get("current_feat_cols")
print("Current-visit features: ")
for col in current_feat_cols:
    print(" -", col)

Current-visit features: 
 - LOG_HOSPITALIZATION_DAYS
 - LOG_DAYS_IN_ICU
 - CHARLSON_INDEX
 - LOG_NUM_DRUGS
 - NUM_PROCEDURES
 - LOG_PARTICIPATION_DAYS
 - HAS_DIABETES
 - HAS_COPD
 - HAS_CONGESTIVE_HF
 - DISCHARGE_LOCATION_POST_ACUTE_CARE
 - DISCHARGE_LOCATION_HOME
 - AGE
 - GENDER_M
 - ADMISSION_TYPE_ELECTIVE
 - ETHNICITY_WHITE
 - ETHNICITY_BLACK
 - ETHNICITY_HISPANIC
 - INSURANCE_MEDICAID
 - INSURANCE_PRIVATE


In [7]:
longitudinal_feat_cols = model_cfg.get("longitudinal_feat_cols")
print("Longitudinal features: ")
for col in longitudinal_feat_cols:
    print(" -", col)

Longitudinal features: 
 - LOG_HOSPITALIZATION_DAYS
 - LOG_DAYS_IN_ICU
 - CHARLSON_INDEX
 - LOG_NUM_DRUGS
 - NUM_PROCEDURES
 - DISCHARGE_LOCATION_POST_ACUTE_CARE
 - ADMISSION_TYPE_ELECTIVE
 - LOG_DAYS_UNTIL_NEXT_HOSPITALIZATION


In [8]:
test_dataset_obj = torch.load(DATASET_TEST_PT_PATH, weights_only=False)
val_dataset_obj = torch.load(DATASET_VAL_PT_PATH, weights_only=False)
train_dataset_obj = torch.load(DATASET_TRAIN_PT_PATH, weights_only=False)
print(f"Loaded dataset_object type: {type(test_dataset_obj)}")

Loaded dataset_object type: <class 'recurrent_health_events_prediction.datasets.HospReadmDataset.HospReadmDataset'>


In [9]:
x_curr_ex, x_past_ex, mask_ex, label_ex = train_dataset_obj[0]
print("Shapes of tensors from dataset:")
print(f"x_curr: {x_curr_ex.shape}")
print(f"x_past: {x_past_ex.shape}")
print(f"mask: {mask_ex.shape}")
print(f"label: {label_ex.shape}")

Shapes of tensors from dataset:
x_curr: torch.Size([19])
x_past: torch.Size([4, 8])
mask: torch.Size([4])
label: torch.Size([])


In [10]:
train_loader = torch.utils.data.DataLoader(
    train_dataset_obj, batch_size=model_cfg["batch_size"], shuffle=False
)
val_loader = torch.utils.data.DataLoader(
    val_dataset_obj, batch_size=model_cfg["batch_size"], shuffle=False
)
test_loader = torch.utils.data.DataLoader(
    test_dataset_obj, batch_size=model_cfg["batch_size"], shuffle=False
)

In [11]:
x_curr, x_past, mask, label = next(iter(train_loader))
print("Shapes of tensors from DataLoader:")
print("x_curr:", x_curr.shape)
print("x_past:", x_past.shape)
print("mask:", mask.shape)
print("label:", label.shape)

Shapes of tensors from DataLoader:
x_curr: torch.Size([32, 19])
x_past: torch.Size([32, 4, 8])
mask: torch.Size([32, 4])
label: torch.Size([32])


In [12]:
# Import model class and instantiate
mod = import_module(MODEL_MODULE)
ModelClass = getattr(mod, MODEL_CLASS_NAME)
model = ModelClass(**MODEL_PARAMS).eval()  # eval mode for graph drawing
print(model.__class__.__name__, "initialized.")

AttentionPoolingNet initialized.


In [34]:
from recurrent_health_events_prediction.training.train_deep_learning_models import train

model, loss_epochs = train(
    train_dataset_obj,
    model_cfg,
    neptune_run=None,
    writer=None,
    val_dataset=val_dataset_obj,
    early_stopping_patience=5,
)

Validation samples: 841

Using model class: AttentionPoolingNet

Model initialized and ready for training.
Model parameters:
  dropout: 0.12112313358911685
  hidden_size_head: 128
  hidden_size_seq: 16
  input_size_curr: 19
  input_size_seq: 8

Starting training...
Max epochs: 100
Number of training samples: 3593
Batch size: 32
Learning rate: 0.008273075604824992
Weight decay: 0.08079331124066194
LR Scheduler: cosine
Optimizer: AdamW
Loss function: BCEWithLogitsLoss

Epoch 1/100, Loss: 0.575262
  -> Val AUROC: 0.618532
  -> Current LR: 0.00827104
Epoch 2/100, Loss: 0.558776
  -> Val AUROC: 0.632068
  -> Current LR: 0.00826492
Epoch 3/100, Loss: 0.553902
  -> Val AUROC: 0.621944
  -> Current LR: 0.00825474
Epoch 4/100, Loss: 0.555393
  -> Val AUROC: 0.636320
  -> Current LR: 0.0082405
Epoch 5/100, Loss: 0.549863
  -> Val AUROC: 0.631536
  -> Current LR: 0.00822221
Epoch 6/100, Loss: 0.546859
  -> Val AUROC: 0.646654
  -> Current LR: 0.00819989
Epoch 7/100, Loss: 0.542981
  -> Val AUROC:

## Helpers

In [22]:
# Try to build a batch (x_current, x_past, mask_past, labels) in a few common cases
def extract_batch(dataset_obj, batch_size):
    """Return a tuple (x_current, x_past, mask_past, labels).
    Supports a Dataset, a tuple of tensors, or a dict.
    """
    if hasattr(dataset_obj, '__getitem__') and hasattr(dataset_obj, '__len__'):
        # Looks like a Dataset
        sample = dataset_obj[0]
        if isinstance(sample, (list, tuple)) and len(sample) >= 4:
            loader = torch.utils.data.DataLoader(dataset_obj, batch_size=batch_size, shuffle=False)
            batch = next(iter(loader))
            return batch  # expect (x_current, x_past, mask_past, labels)
        else:
            raise ValueError("Dataset sample does not look like (x_current, x_past, mask_past, labels)")

## Global Feature Importance

In [35]:
from recurrent_health_events_prediction.model.explainers import global_feature_importance

curr_attr_df, past_attr_df, mean_abs_time = (
    global_feature_importance(model, train_loader, train_loader, current_feat_cols, longitudinal_feat_cols, device)
)

mean_curr shape: torch.Size([19])
mean_past shape: torch.Size([8])
has_median: True


In [31]:
curr_attr_df.columns

Index(['feature', 'attribution_activity', 'attribution_direction'], dtype='object')

In [36]:
from recurrent_health_events_prediction.training.utils_deep_learning import (
    plot_feature_attributions,
)

plot_feature_attributions(
    curr_attr_df,
    title="Current Visit Features - Global Attributions",
    feature_col="feature",
    attr_col="attribution_activity"
)

In [18]:
plot_feature_attributions(
    curr_attr_df,
    title="Current Visit Features - Global Attributions (Direction)",
    feature_col="feature",
    attr_col="attribution_direction",
    top_k=None
)

In [19]:
from recurrent_health_events_prediction.training.utils_deep_learning import (
    plot_feature_attributions,
)

plot_feature_attributions(
    past_attr_df,
    title="Past Visit Features - Global Attributions",
    feature_col="feature",
    attr_col="attribution_activity",
    top_k=10,
)

In [20]:
import numpy as np
import plotly.graph_objects as go

x = np.arange(1, len(mean_abs_time) + 1)

fig = go.Figure(go.Scatter(x=x, y=mean_abs_time, mode="lines+markers", marker=dict(size=6)))
fig.update_layout(
    title="Importance over time",
    xaxis_title="Step",
    yaxis_title="Mean absolute importance",
    template="plotly_white",
    width=800,
    height=400,
)
fig.show()

## Global Feature Importance - 4 past hospitalizations

In [30]:
from torch.utils.data import Subset

def filter_full_mask(dataset):
    indices_full_mask = []

    for idx, sample in enumerate(dataset):
        try:
            x_curr, x_past, mask, label = sample
        except Exception:
            # skip samples with unexpected format
            continue

        # normalize mask shape to [T]
        if mask.dim() == 3 and mask.shape[-1] == 1:
            mask = mask.squeeze(-1)
        mask = mask.bool()

        # check if all time steps are valid
        if mask.numel() > 0 and mask.all():
            indices_full_mask.append(idx)

    print(f"Found {len(indices_full_mask)} samples with all-true mask.")
    if indices_full_mask:
        print("Sample indices (preview):", indices_full_mask[:20])

    # create and return filtered subset
    return Subset(dataset, indices_full_mask)


In [24]:
full_mask_test_dataset = filter_full_mask(test_dataset_obj)
full_mask_train_dataset = filter_full_mask(train_dataset_obj)

Found 144 samples with all-true mask.
Sample indices (preview): [7, 8, 36, 48, 49, 63, 78, 91, 126, 157, 161, 162, 171, 172, 173, 202, 203, 204, 205, 206]
Found 600 samples with all-true mask.
Sample indices (preview): [43, 44, 45, 46, 89, 90, 91, 92, 110, 111, 112, 113, 114, 115, 155, 156, 178, 188, 209, 210]


In [31]:
full_mask_test_loader = torch.utils.data.DataLoader(
    full_mask_test_dataset, batch_size=model_cfg["batch_size"], shuffle=False
)
full_mask_train_dataset_loader = torch.utils.data.DataLoader(
    full_mask_train_dataset, batch_size=model_cfg["batch_size"], shuffle=False
)

In [32]:
curr_attr_df, past_attr_df, mean_abs_time = (
    global_feature_importance(model, train_loader, val_loader, current_feat_cols, longitudinal_feat_cols, device)
)

mean_curr shape: torch.Size([19])
mean_past shape: torch.Size([8])
has_median: True


In [33]:
from recurrent_health_events_prediction.model_selection.deep_learning.utils import plot_attribution_over_time


plot_attribution_over_time(mean_abs_time, title="Absolute IG Attributions - Time Indexes")

## Feature Importance per Sample

In [4]:
import os
import json
import torch

if os.path.exists(TRAINING_DATA_STATS_PATH):
    with open(TRAINING_DATA_STATS_PATH, "r") as f:
        raw = json.load(f)
    stats = {}
    for k, v in raw.items():
        # convert lists back to tensors when appropriate
        if isinstance(v, list):
            stats[k] = torch.tensor(v)
        else:
            stats[k] = v
    print(f"Loaded training stats from: {TRAINING_DATA_STATS_PATH}")
else:
    print(f"No training stats file found at: {TRAINING_DATA_STATS_PATH}. Will compute stats.")

Loaded training stats from: /workspaces/msc-thesis-recurrent-health-modeling/_models/mimic/deep_learning/training_stats.json


In [16]:
# get a batch to explain
batch_example = extract_batch(test_dataset_obj, batch_size=8)
x_curr_b, x_past_b, mask_b, _ = batch_example

# (Optional) layer to split history vs current — for your AttentionPoolingNet:
fc1_layer = model.classifier_head[0]  # nn.Linear

In [17]:
model.forward(x_curr_b,
    x_past_b,
    mask_b,
)

(tensor([ 0.0090,  0.0241, -0.1159, -0.0580,  0.0462, -0.1198, -0.0118, -0.1127],
        grad_fn=<SqueezeBackward1>),
 tensor([[0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000],
         [1.0000, 0.0000, 0.0000, 0.0000],
         [0.3988, 0.6012, 0.0000, 0.0000],
         [0.4077, 0.3661, 0.2262, 0.0000],
         [0.2530, 0.2551, 0.2342, 0.2577]], grad_fn=<IndexPutBackward0>))

In [18]:
from recurrent_health_events_prediction.model.explainers import explain_deep_learning_model_feat

df_curr, df_past, df_split  = explain_deep_learning_model_feat(
    model,
    x_curr_b,
    x_past_b,
    mask_b,
    current_feat_cols,
    longitudinal_feat_cols,
    layer_for_split=fc1_layer,
    baseline_strategy="means",
    stats=stats,
    n_steps=64,
    internal_batch_size=16,
)

In [19]:
df_curr.sort_values(by="sample_idx", ascending=True)

,sample_idx,feature,attribution,attribution_signed
0,0,LOG_HOSPITALIZATION_DAYS,0.001871,0.001871
1,0,LOG_DAYS_IN_ICU,0.019085,0.019085
2,0,CHARLSON_INDEX,0.018571,0.018571
3,0,LOG_NUM_DRUGS,0.003117,0.003117
4,0,NUM_PROCEDURES,0.003849,0.003849
...,...,...,...,...
147,7,ETHNICITY_WHITE,0.004308,-0.004308
148,7,ETHNICITY_BLACK,0.000605,-0.000605
149,7,ETHNICITY_HISPANIC,0.001917,-0.001917
150,7,INSURANCE_MEDICAID,0.001491,0.001491


In [20]:
import pandas as pd

mean_curr_df = pd.DataFrame(
    {
        "feature": current_feat_cols,
        "mean_value": stats["mean_curr"].cpu().numpy(),
    }
)

mean_curr_df

,feature,mean_value
0,LOG_HOSPITALIZATION_DAYS,0.001942
1,LOG_DAYS_IN_ICU,0.013992
2,CHARLSON_INDEX,0.015768
3,LOG_NUM_DRUGS,0.014111
4,NUM_PROCEDURES,-0.015215
5,LOG_PARTICIPATION_DAYS,0.031994
6,HAS_DIABETES,0.450383
7,HAS_COPD,0.368516
8,HAS_CONGESTIVE_HF,0.501804
9,DISCHARGE_LOCATION_POST_ACUTE_CARE,0.420613


In [21]:
feature_values_df = pd.DataFrame({
    "sample_idx": [i for i in range(len(x_curr_b)) for _ in current_feat_cols],
    "feature": current_feat_cols * len(x_curr_b),
    "value": x_curr_b.flatten().detach().numpy(),
})

In [22]:
merged_df = pd.merge(df_curr, feature_values_df, on=["sample_idx", "feature"])
merged_df = pd.merge(merged_df, mean_curr_df, on="feature", suffixes=("", "_mean"))
merged_df.head(21)

,sample_idx,feature,attribution,attribution_signed,value,mean_value
0,0,LOG_HOSPITALIZATION_DAYS,0.001871,0.001871,0.055471,0.001942
1,0,LOG_DAYS_IN_ICU,0.019085,0.019085,-0.798306,0.013992
2,0,CHARLSON_INDEX,0.018571,0.018571,-0.785789,0.015768
3,0,LOG_NUM_DRUGS,0.003117,0.003117,0.640240,0.014111
4,0,NUM_PROCEDURES,0.003849,0.003849,1.175824,-0.015215
5,0,LOG_PARTICIPATION_DAYS,0.027145,0.027145,-1.238572,0.031994
6,0,HAS_DIABETES,0.026185,-0.026185,0.000000,0.450383
7,0,HAS_COPD,0.018397,-0.018397,1.000000,0.368516
8,0,HAS_CONGESTIVE_HF,0.010578,0.010578,0.000000,0.501804
9,0,DISCHARGE_LOCATION_POST_ACUTE_CARE,0.007619,0.007619,0.000000,0.420613


In [24]:
merged_df

,sample_idx,feature,attribution,attribution_signed,value,mean_value
0,0,LOG_HOSPITALIZATION_DAYS,0.001871,0.001871,0.055471,0.001942
1,0,LOG_DAYS_IN_ICU,0.019085,0.019085,-0.798306,0.013992
2,0,CHARLSON_INDEX,0.018571,0.018571,-0.785789,0.015768
3,0,LOG_NUM_DRUGS,0.003117,0.003117,0.640240,0.014111
4,0,NUM_PROCEDURES,0.003849,0.003849,1.175824,-0.015215
...,...,...,...,...,...,...
147,7,ETHNICITY_WHITE,0.004308,-0.004308,1.000000,0.665088
148,7,ETHNICITY_BLACK,0.000605,-0.000605,0.000000,0.230492
149,7,ETHNICITY_HISPANIC,0.001917,-0.001917,0.000000,0.041498
150,7,INSURANCE_MEDICAID,0.001491,0.001491,0.000000,0.112765


In [ ]:
merged_df.to_csv("local_attributions_current_features.csv", index=False)

In [86]:
sign_consistency = merged_df.groupby('feature')['attribution'].apply(lambda x: (x > 0).mean())
print(sign_consistency.sort_values())

feature
ADMISSION_TYPE_ELECTIVE               0.000
ETHNICITY_BLACK                       0.000
ETHNICITY_HISPANIC                    0.125
HAS_CONGESTIVE_HF                     0.125
HAS_COPD                              0.125
INSURANCE_MEDICAID                    0.125
LOG_PARTICIPATION_DAYS                0.250
HAS_DIABETES                          0.500
LOG_NUM_DRUGS                         0.500
NUM_PROCEDURES                        0.500
DISCHARGE_LOCATION_HOME               0.500
AGE                                   0.625
CHARLSON_INDEX                        0.625
INSURANCE_PRIVATE                     0.750
LOG_HOSPITALIZATION_DAYS              0.750
ETHNICITY_WHITE                       0.750
LOG_DAYS_IN_ICU                       0.875
DISCHARGE_LOCATION_POST_ACUTE_CARE    1.000
GENDER_M                              1.000
Name: attribution, dtype: float64


In [ ]:
df_past.sort_values(by="importance", ascending=False).head(10)

In [28]:
df_split.iloc[0].to_dict()

{'sample_idx': 0.0,
 'past_attribution': 0.08032890434862353,
 'current_attribution': 0.33738889031117253}